In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

In [19]:
# 归一化将数据转化到 0-1 之间的范围
# sl:satisfaction_level
# le:last_evaluation
# npr:number_project
# amh:average_monthly_hours
# tsc:time_spend_company
# wa:Work_accident
# pl5:promotion_last_5years
# False:MinMaxScaler
# True:StandardScaler
# 将离散值数值化
# dp:department 
# slr:salary
# False:LabelEncoding
# True:OneHotEncoder
# 降维
# lower_d = False  不降维
# ld_n = 1 下降的维度
def hr_preprocessing(sl=False, le=False, npr=False, amh=False, tsc=False, wa=False, pl5=False, dp=False, slr=False, lower_d = False, ld_n = 1):
    df = pd.read_csv('./data/HR.csv')
    # 1.清洗数据
    df = df.dropna(subset=['satisfaction_level', 'last_evaluation'])
    df = df[df['satisfaction_level']<=1][df['salary']!='nme']
    # 2.得到标注
    label = df["left"]
    df = df.drop('left', axis=1)
    # 3.特征选择
    
    # 4.特征处理
    scaler_lst = [sl, le, npr, amh, tsc, wa, pl5]
    column_lst = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_monthly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
        else:
            df[column_lst[i]]=StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
    # # 将离散值数值化
    scaler_lst = [dp, slr]
    column_lst = ["department", "salary"]
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            if column_lst[i] == "salary":
                df[column_lst[i]] = [map_salary(s) for s in df['salary'].values]
            else:
                df[column_lst[i]] = LabelEncoder().fit_transform(df[column_lst[i]])
            df[column_lst[i]] = MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
        else:
            # OneHotEncoder
            df = pd.get_dummies(df, columns=[column_lst[i]])
    
    # 降维
    if lower_d:
        # n_components 不能大于类的个数
        # return LinearDiscriminantAnalysis(n_components=ld_n)
        
        # PCA n_components 不受限制
        return PCA(n_components=ld_n).fit_transform(df.values)
        
    
    return df, label

d = dict([('low', 0), ('medium', 1), ('high', 2)])
def map_salary(s):
    return d.get(s, 0)
    
def main():
    print(hr_preprocessing(lower_d = False, ld_n = 3))

if __name__=="__main__":
    main()

(       satisfaction_level  last_evaluation  number_project  \
0                0.318681         0.265625             0.0   
1                0.780220         0.781250             0.6   
2                0.021978         0.812500             1.0   
3                0.692308         0.796875             0.6   
4                0.307692         0.250000             0.0   
...                   ...              ...             ...   
14994            0.340659         0.328125             0.0   
14995            0.307692         0.187500             0.0   
14996            0.307692         0.265625             0.0   
14997            0.021978         0.937500             0.8   
14998            0.307692         0.250000             0.0   

       average_monthly_hours  time_spend_company  Work_accident  \
0                   0.285047               0.125            0.0   
1                   0.775701               0.500            0.0   
2                   0.822430               0.250     